# BLG-407 Makine Öğrenmesi - Proje 3  
## Çoklu Doğrusal Regresyon + Flask Arayüz Uygulaması

**Ad:** İbrahim
**Soyad:** Çerkezoğlu
**Numara:** 2112721046
**GitHub Repo:** https://github.com/ibrahimcerkezoglu/CarPricePrediction-ML-Flask


## 1. Veri Setinin Tanıtılması

Bu projede araç fiyatlarının tahmin edilmesi amaçlanmıştır. Veri setinde araçlara ait teknik ve kategorik bilgiler yer almaktadır. İlk olarak veri seti okunmuş ve genel yapısı incelenmiştir.

Aşağıdaki işlemler yapılmıştır:

- İlk 5 satır görüntülenmiştir.  
- Sütun veri tipleri incelenmiştir.  
- Sayısal özet istatistikleri çıkarılmıştır (mean, std, min, max).  
- Eksik veri analizi yapılmıştır.

Bu aşama veri setinin yapısını anlamak için gereklidir.


In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("Data/car_price.csv")
df.head()


,CarName,carbody,drivewheel,enginelocation,fueltype,aspiration,doornumber,cylindernumber,enginetype,fuelsystem,...,curbweight,enginesize,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,alfa-romero giulia,convertible,rwd,front,gas,std,two,four,dohc,mpfi,...,2548,130,3.47,2.68,9.0,111,5000,21,27,13495.0
1,alfa-romero stelvio,convertible,rwd,front,gas,std,two,four,dohc,mpfi,...,2548,130,3.47,2.68,9.0,111,5000,21,27,16500.0
2,alfa-romero Quadrifoglio,hatchback,rwd,front,gas,std,two,six,ohcv,mpfi,...,2823,152,2.68,3.47,9.0,154,5000,19,26,16500.0
3,audi 100 ls,sedan,fwd,front,gas,std,four,four,ohc,mpfi,...,2337,109,3.19,3.40,10.0,102,5500,24,30,13950.0
4,audi 100ls,sedan,4wd,front,gas,std,four,five,ohc,mpfi,...,2824,136,3.19,3.40,8.0,115,5500,18,22,17450.0


In [2]:
df.info()
df.describe()
df.isna().sum()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 25 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CarName           210 non-null    object 
 1   carbody           210 non-null    object 
 2   drivewheel        210 non-null    object 
 3   enginelocation    210 non-null    object 
 4   fueltype          210 non-null    object 
 5   aspiration        210 non-null    object 
 6   doornumber        210 non-null    object 
 7   cylindernumber    210 non-null    object 
 8   enginetype        210 non-null    object 
 9   fuelsystem        210 non-null    object 
 10  symboling         210 non-null    int64  
 11  wheelbase         210 non-null    float64
 12  carlength         210 non-null    float64
 13  carwidth          210 non-null    float64
 14  carheight         210 non-null    float64
 15  curbweight        210 non-null    int64  
 16  enginesize        210 non-null    int64  
 1

CarName             0
carbody             0
drivewheel          0
enginelocation      0
fueltype            0
aspiration          0
doornumber          0
cylindernumber      0
enginetype          0
fuelsystem          0
symboling           0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginesize          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

## 2. Öznitelik (Feature) Seçimi

Projede en fazla **10 adet öznitelik** kullanılması gerektiğinden, veri setindeki tüm özellikler incelenmiş ve aşağıdaki 10 öznitelik modele dahil edilmiştir:

- wheelbase  
- carlength  
- carwidth  
- curbweight  
- enginesize  
- horsepower  
- citympg  
- highwaympg  
- carbody (kategorik)  
- drivewheel (kategorik)

Bu özellikler aracın fiziksel yapısını ve performansını temsil ettiği için fiyat üzerinde önemli etkiye sahiptir.


In [3]:
selected_features = [
    'wheelbase', 'carlength', 'carwidth', 'curbweight',
    'enginesize', 'horsepower', 'citympg', 'highwaympg',
    'carbody', 'drivewheel'
]

target = 'price'

data = df[selected_features + [target]].copy()
data.head()


,wheelbase,carlength,carwidth,curbweight,enginesize,horsepower,citympg,highwaympg,carbody,drivewheel,price
0,88.6,168.8,64.1,2548,130,111,21,27,convertible,rwd,13495.0
1,88.6,168.8,64.1,2548,130,111,21,27,convertible,rwd,16500.0
2,94.5,171.2,65.5,2823,152,154,19,26,hatchback,rwd,16500.0
3,99.8,176.6,66.2,2337,109,102,24,30,sedan,fwd,13950.0
4,99.4,176.6,66.4,2824,136,115,18,22,sedan,4wd,17450.0


In [4]:
num_cols = ['wheelbase', 'carlength', 'carwidth', 'curbweight',
             'enginesize', 'horsepower', 'citympg', 'highwaympg']

cat_cols = ['carbody', 'drivewheel']


## 3. Sayısal ve Kategorik Değişkenlerin Ayrılması

Modelin doğru şekilde işlenebilmesi için değişkenler iki gruba ayrılmıştır:

- **Sayısal değişkenler:** Ölçekleme uygulanmıştır (StandardScaler).
- **Kategorik değişkenler:** One-Hot Encoding uygulanmıştır. `drop='first'` kullanılarak dummy tuzağı engellenmiştir.

Bu adım çoklu doğrusal regresyon modelleri için kritik öneme sahiptir.


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

X = data[selected_features]
y = data[target]

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(drop='first'), cat_cols)
    ]
)


## 4. Veri Ön İşleme Pipeline Yapısı

Aşağıdaki işlemler tek bir pipeline içerisinde birleştirilmiştir:

- Sayısal veriler → StandardScaler ile ölçeklendi  
- Kategorik veriler → OneHotEncoder ile sayısal forma dönüştürüldü  
- LinearRegression modeli aynı pipeline içinde eğitildi  

Bu yapı, modelin Flask ortamında da tek adımda çalışmasını sağlamaktadır.


In [9]:
import statsmodels.api as sm

# Preprocessing'i fit-transform edelim
X_prep = preprocessor.fit_transform(X)

# OHE'den gelen isimleri al
ohe_names = preprocessor.named_transformers_['cat'].get_feature_names_out(cat_cols)
all_features = num_cols + list(ohe_names)

for i, name in enumerate(all_features):
    print(f"x{i+1} = {name}")

# OLS için sabit terim ekle
X_ols = sm.add_constant(X_prep)

model_ols = sm.OLS(y, X_ols).fit()
print(model_ols.summary())


x1 = wheelbase
x2 = carlength
x3 = carwidth
x4 = curbweight
x5 = enginesize
x6 = horsepower
x7 = citympg
x8 = highwaympg
x9 = carbody_hardtop
x10 = carbody_hatchback
x11 = carbody_sedan
x12 = carbody_wagon
x13 = drivewheel_fwd
x14 = drivewheel_rwd
                            OLS Regression Results                            
Dep. Variable:                  price   R-squared:                       0.847
Model:                            OLS   Adj. R-squared:                  0.836
Method:                 Least Squares   F-statistic:                     77.22
Date:                Wed, 10 Dec 2025   Prob (F-statistic):           1.56e-71
Time:                        17:00:18   Log-Likelihood:                -1985.4
No. Observations:                 210   AIC:                             4001.
Df Residuals:                     195   BIC:                             4051.
Df Model:                          14                                         
Covariance Type:            nonrobust    

Bu bölümde Backward Elimination uygulanmıştır.
p-value > 0.05 olan özellikler sırasıyla çıkarılarak final model elde edilmiştir.


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

regressor = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LinearRegression())
])

regressor.fit(X_train, y_train)
y_pred = regressor.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)

r2, mae, mse


(0.7479249940266572, 3383.4579800927118, 18991807.27401704)

### 5. OLS Regresyon Sonuçlarının Değerlendirilmesi

OLS çıktısı incelendiğinde modelin istatistiksel olarak anlamlı olduğu görülmüştür:

- **R² ≈ 0.84** → Model fiyat değişkeninin büyük bir kısmını açıklamaktadır.
- **F-istatistiği p-value < 0.001** → Model genel olarak anlamlıdır.
- p > 0.05 olan değişkenler tespit edilerek Backward Elimination uygulanmıştır.

Bu analiz sonucunda aşağıdaki değişkenler modelden çıkarılmıştır:

- curbweight  
- carbody_sedan  
- drivewheel_fwd  
- carlength  

Final model istatistiksel olarak daha güçlü hale getirilmiştir.


## 6. Backward Elimination (Geriye Doğru Eleme) Yöntemi

Bu aşamada modelin istatistiksel olarak anlamlı özellikleri belirlenmiştir. Tüm değişkenlerle model kurulmuş, OLS regresyonu ile p-value değerleri incelenmiş ve p > 0.05 olan özellikler sırayla çıkarılarak süreç tekrarlanmıştır.

### Sürecin Özeti:

1. İlk model tüm değişkenlerle kurulmuştur.  
2. p > 0.05 olan en büyük p-value’lı değişken çıkartılmıştır.  
3. Model tekrar kurulmuş ve p-value analizine devam edilmiştir.  
4. Tüm anlamsız değişkenler çıkarılana kadar süreç tekrarlanmıştır.

### Çıkarılan Değişkenler Sırasıyla:
- curbweight  
- carbody_sedan  
- drivewheel_fwd  
- carlength  
- highwaympg  
- carbody_hardtop  
- carbody_wagon  
- citympg  
- wheelbase  

### Final Modelde Kalan Anlamlı Değişkenler:
- carwidth  
- enginesize  
- horsepower  
- drivewheel_rwd  

### Sonuç:
Final çoklu doğrusal regresyon modeli istatistiksel olarak anlamlı olup fiyatı açıklamada güçlü bir performans göstermektedir.


In [8]:
import pickle

with open("model.pkl", "wb") as f:
    pickle.dump(regressor, f)


## 7. Modelin Kaydedilmesi (.pkl)

Eğitilen model Flask uygulamasında kullanılmak üzere `model.pkl` dosyasına kaydedilmiştir.  
Bu yöntem sayesinde model tekrar eğitilmeden hızlı şekilde yüklenebilir.


## 8. Flask Arayüz Uygulaması

Model Flask tabanlı bir web arayüzüne entegre edilmiştir.  
Kullanıcıdan aşağıdaki özellikler alınmakta ve model fiyat tahmini üretmektedir:

- wheelbase  
- carlength  
- carwidth  
- curbweight  
- enginesize  
- horsepower  
- citympg  
- highwaympg  
- carbody  
- drivewheel  

Model arka planda pipeline üzerinden otomatik olarak ölçekleme ve kodlama işlemlerini uygulamaktadır.

Aşağıda Flask arayüzünden örnek bir ekran görüntüsü bulunmaktadır:
